In [2]:
import os
import urllib.request
import py7zr
import pandas as pd
from sqlalchemy import create_engine
import shutil
import unicodedata

In [49]:

# Solicitar ao usuário o número do mês e o ano desejados
mes = input("Digite o número do mês, sempre com dois dígitos (ex: 01 para janeiro): ")
ano = input("Digite o ano (ex: 2023): ")

# URL do arquivo no FTP
file_url = f"ftp://ftp.mtps.gov.br/pdet/microdados/NOVO%20CAGED/{ano}/{ano}{mes}/CAGEDMOV{ano}{mes}.7z"

# Diretório local para salvar o arquivo baixado
local_dir = "C:/Users/ricardo/Downloads"

# Nome do arquivo local
local_file = f"CAGEDMOV{ano}{mes}.7z"

# Baixar o arquivo do FTP
urllib.request.urlretrieve(file_url, filename=os.path.join(local_dir, local_file))

print("Download concluído.")

Digite o número do mês, sempre com dois dígitos (ex: 01 para janeiro): 12
Digite o ano (ex: 2023): 2021
Download concluído.


In [50]:
# Diretório local de download
download_dir = "C:/Users/ricardo/Downloads"

# Diretório para salvar os arquivos descompactados
extracted_dir = "C:/Users/ricardo/Desktop/CAGED/DADOS BRUTOS"

# Nome do arquivo compactado
zip_file = f"CAGEDMOV{ano}{mes}.7z"

# Caminho completo para o arquivo compactado
zip_path = os.path.join(download_dir, zip_file)

# Verificar se o arquivo compactado existe na pasta de downloads
if os.path.exists(zip_path):
    # Descompactar o arquivo
    with py7zr.SevenZipFile(zip_path, mode='r') as z:
        z.extractall(extracted_dir)
    print(f"Arquivo CAGEDMOV{ano}{mes} descompactados e salvos em:", extracted_dir)
else:
    print("O arquivo compactado não foi encontrado na pasta de downloads.")

Arquivo CAGEDMOV202112 descompactados e salvos em: C:/Users/ricardo/Desktop/CAGED/DADOS BRUTOS


In [19]:
#Descobrindo as colunas presentes no Arqvuio .txt
import pandas as pd

# Ler as primeiras linhas do arquivo .txt para descobrir as colunas
n_linhas_para_leitura = 5  # Defina o número de linhas a serem lidas para inferir as colunas
with open(arquivo_descompactado, 'r', encoding='utf-8') as f:
    colunas = pd.read_csv(f, nrows=n_linhas_para_leitura, delimiter=';').columns.tolist()

# Imprimir as colunas descobertas
print("Colunas presentes no arquivo:")
print(colunas)

Colunas presentes no arquivo:
['competênciamov', 'região', 'uf', 'município', 'seção', 'subclasse', 'saldomovimentação', 'cbo2002ocupação', 'categoria', 'graudeinstrução', 'idade', 'horascontratuais', 'raçacor', 'sexo', 'tipoempregador', 'tipoestabelecimento', 'tipomovimentação', 'tipodedeficiência', 'indtrabintermitente', 'indtrabparcial', 'salário', 'tamestabjan', 'indicadoraprendiz', 'origemdainformação', 'competênciadec', 'indicadordeforadoprazo', 'unidadesaláriocódigo', 'valorsaláriofixo']


In [23]:
#remover os caracteres "especiais" das colunas antes de criar o banco de dados
# Função para remover caracteres especiais e espaços dos nomes das colunas
def limpar_nome_coluna(nome):
    nome_normalizado = unicodedata.normalize('NFKD', nome).encode('ascii', 'ignore').decode('utf-8')
    return nome_normalizado.replace(' ', '_')

# Lista de nomes de colunas limpos
colunas_limpos = [limpar_nome_coluna(coluna) for coluna in colunas]
print(colunas_limpos)


['competenciamov', 'regiao', 'uf', 'municipio', 'secao', 'subclasse', 'saldomovimentacao', 'cbo2002ocupacao', 'categoria', 'graudeinstrucao', 'idade', 'horascontratuais', 'racacor', 'sexo', 'tipoempregador', 'tipoestabelecimento', 'tipomovimentacao', 'tipodedeficiencia', 'indtrabintermitente', 'indtrabparcial', 'salario', 'tamestabjan', 'indicadoraprendiz', 'origemdainformacao', 'competenciadec', 'indicadordeforadoprazo', 'unidadesalariocodigo', 'valorsalariofixo']


In [24]:
# Gerar a query SQL para criar a tabela no PostgreSQL (se necessário)
# Lista de tipos de dados padrão para todas as colunas (você pode ajustar conforme necessário)
tipos_dados = ['integer', 'integer', 'integer', 'integer', 'character varying(255)', 'integer', 'integer', 'integer', 'integer', 'integer', 'integer', 'character varying(255)', 'integer', 'integer', 'integer', 'integer', 'integer', 'integer', 'integer', 'integer', 'character varying(255)', 'integer', 'integer', 'integer', 'integer', 'integer', 'integer', 'character varying(255)']

# Gerar a query SQL para criar a tabela no PostgreSQL
query_criar_tabela = f"""
-- DROP TABLE IF EXISTS "{ano}"."{mes}-{ano}";

CREATE TABLE IF NOT EXISTS "{ano}"."{mes}-{ano}"
(
    {', '.join([f'"{coluna}" {tipo_dado}' for coluna, tipo_dado in zip(colunas_limpos, tipos_dados)])}
)

TABLESPACE pg_default;

ALTER TABLE IF EXISTS "{ano}"."{mes}-{ano}"
    OWNER to postgres;
"""

print("Query SQL para criar a tabela no PostgreSQL:")
print(query_criar_tabela)



Query SQL para criar a tabela no PostgreSQL:

-- DROP TABLE IF EXISTS "2023"."01-2023";

CREATE TABLE IF NOT EXISTS "2023"."01-2023"
(
    "competenciamov" integer, "regiao" integer, "uf" integer, "municipio" integer, "secao" character varying(255), "subclasse" integer, "saldomovimentacao" integer, "cbo2002ocupacao" integer, "categoria" integer, "graudeinstrucao" integer, "idade" integer, "horascontratuais" character varying(255), "racacor" integer, "sexo" integer, "tipoempregador" integer, "tipoestabelecimento" integer, "tipomovimentacao" integer, "tipodedeficiencia" integer, "indtrabintermitente" integer, "indtrabparcial" integer, "salario" character varying(255), "tamestabjan" integer, "indicadoraprendiz" integer, "origemdainformacao" integer, "competenciadec" integer, "indicadordeforadoprazo" integer, "unidadesalariocodigo" integer, "valorsalariofixo" character varying(255)
)

TABLESPACE pg_default;

ALTER TABLE IF EXISTS "2023"."01-2023"
    OWNER to postgres;



In [18]:
# Caminho completo para o arquivo descompactado
arquivo_descompactado = os.path.join(extracted_dir, f'CAGEDMOV{ano}{mes}.txt')

# Substitua as barras invertidas duplicadas por uma única barra invertida
arquivo_descompactado = arquivo_descompactado.replace('\\', '/')

# Query SQL para carregar os dados do arquivo descompactado para o PostgreSQL
query_sql = f"""
COPY {nome_tabela} (competenciamov, regiao, uf, municipio, secao, subclass, saldomovimentacao, cbo2002ocupacao, categoria, graudeinstrucao, idade, horascontratuais, racacor, sexo, tipoempregador, tipoestabelecimento, tipomovimentacao, tipodedeficiencia, indtrabintermitente, indtrabparcial, salario, tamestabjan, indicadoraprendiz, origemdainformacao, competenciadec, indicadordeforadoprazo, unidadesalariocodigo, valorsalariofixo) 
FROM '{arquivo_descompactado}' 
WITH (FORMAT CSV, HEADER, DELIMITER ';', ENCODING 'UTF8', QUOTE '"', ESCAPE '''');
"""

print("Query SQL criada:")
print(query_sql)


Query SQL criada:

COPY "2023"."01-2023" (competenciamov, regiao, uf, municipio, secao, subclass, saldomovimentacao, cbo2002ocupacao, categoria, graudeinstrucao, idade, horascontratuais, racacor, sexo, tipoempregador, tipoestabelecimento, tipomovimentacao, tipodedeficiencia, indtrabintermitente, indtrabparcial, salario, tamestabjan, indicadoraprendiz, origemdainformacao, competenciadec, indicadordeforadoprazo, unidadesalariocodigo, valorsalariofixo) 
FROM 'C:/Users/ricardo/Desktop/CAGED/DADOS BRUTOS/CAGEDMOV202301.txt' 
WITH (FORMAT CSV, HEADER, DELIMITER ';', ENCODING 'UTF8', QUOTE '"', ESCAPE '''');

